# <center>Atelier Scraping </center>

<div style="text-align: center;">
    <img src="../images/scrappeur.png" width="600" height="300">
</div>
___

## Introduction

Commençons par mettre en place les bases du scraping. Scraper, c'est savoir lire ce qui se cache derrière un site. Avec un simple clic droit et inspecter l'élément, on arrive assez simplement à accéder au code HTML de la page.

<div style="text-align: center;">
    <img src="../images/du site au html.png" width="800" height="400">
</div>

Voici un petit [Formulaire HTML](formulaire_html.md). Il permet rapidemment d'apprendre ou se remémorer les balises principales permettant de lire du HTML et d'ainsi repérer les différents éléments d'une page web.

Ensuite, il faut choisir un outil de scraping : Selenium ou BeautifulSoup.

- Selenium est utile pour les pages web dynamiques où le contenu est généré via JavaScript, nécessitant des interactions utilisateur telles que les clics, le défilement, ou la saisie de texte.

- BeautifulSoup est une bibliothèque Python utilisée principalement pour parser (analyser) des documents HTML et XML. Elle est utile pour extraire des données structurées à partir de pages web statiques.

De manière générale, on préfèrera utiliser Selenium qui permet plus d'actions, mais beautifulsoup reste une option importante. Dans, cette activité on présente donc les deux modules pour se faire la main.

___

## Activité 1a : Récupérer une énigme du Professeur Layton avec Selenium

Nous allons voir comment nous pouvons simplement récupérer les éléments principaux d'une page wiki pour s'en faire une base de données. Pour cela, nous allons nous connecter sur https://professeur-layton.fandom.com/fr/wiki/La_travers%C3%A9e_(1). 

L'idée dans un premier temps sera de récupérer :
- le titre,
- le numéro de l'énigme,
- l'énoncé,
- la solution.

###### Import des modules python

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

###### Lancer l'ouverture du navigateur et la connexion à l'énigme

In [2]:
# Mise en place du driver chrome
service = Service(executable_path=ChromeDriverManager().install())
# Ajouter des options 
chrome_options = Options()
# Désactiver la propriété qui révèle le contrôle par l'automatisation
chrome_options.add_argument('--disable-blink-features=AutomationControlled')  
# On lance Chrome en fournissant le driver et en renseignant les options
driver = webdriver.Chrome(service=service, options=chrome_options)
# On renseigne l'URL que l'on veut scraper
url = "https://professeur-layton.fandom.com/fr/wiki/La_travers%C3%A9e_(1)"
driver.get(url)

###### Récupération astucieuse des éléments Web

C'est ici que l'on introduit les différents sélecteurs d'éléments Web : 

- ID = "id"
- NAME = "name"
- XPATH = "xpath"
- LINK_TEXT = "link text"
- PARTIAL_LINK_TEXT = "partial link text"
- TAG_NAME = "tag name"
- CLASS_NAME = "class name"
- CSS_SELECTOR = "css selector"

In [7]:
# Récupération du titre :
title = driver.find_element(By.XPATH ,'//*[@id="firstHeading"]')

# Récupération du numéro de l'énigme :
num_enigme = driver.find_element(By.XPATH ,'//*[@id="mw-content-text"]/div/div[1]/div[2]/table/tbody/tr[4]/td')

# Récupération de l'énoncé de l'énigme :
enonce = driver.find_element(By.XPATH ,'//*[@id="Énoncé"]')
enigme_enonce = enonce.find_elements(By.XPATH, '//h2[span[@id="Énoncé"]]/following-sibling::p | //h2[span[@id="Énoncé"]]/following-sibling::ul')

# Récupération de la réponse :
reponse = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/p[8]')

###### Lecture des éléments Web

In [8]:
title = title.text
num_enigme = num_enigme.text
enigme_enonce = [elem.text for elem in enigme_enonce]
enigme_enonce = "\n".join(enigme_enonce)
reponse = reponse.text

###### Stockage dans un DataFrame

In [19]:
# Initialiser le dictionnaire de listes
data = {
    'title': title,
    'number': num_enigme,
    'description': enigme_enonce,
    'solution': reponse
}
# Afficher le dictionnaire
print(data)

{'title': 'La traversée (1)', 'number': '007', 'description': "Vous devez amener les trois loups et les trois poussins sur l'autre rive. Attention à bien respecter les règles suivantes\xa0:\n\nLe radeau ne peut accueillir plus de deux animaux.\nVous ne pouvez pas déplacer le radeau s'il est vide.\nS'il y a plus de loups que de poussins sur une des rives, les loups mangeront les pauvres volatiles sans défense et il vous faudra recommencer depuis le début.\nVous n'avez pas de limite de déplacement, mais il est possible de résoudre cette énigme en 11 déplacements.\n", 'solution': 'Beau travail\xa0!\n Cette énigme peut être résolue en 11 déplacements. Combien en avez-vous fait\xa0?\n Il existe de nombreuses variations de ce problème. On a retrouvé de telles énigmes dans des écrits vieux de plus de mille ans.\n'}


___

## Activité 1b : Récupérer une énigme du Professeur Layton avec BeautifulSoup

###### Import des modules python

In [13]:
# Import des bibliothèques
from bs4 import BeautifulSoup
import random
import requests
import re

###### Lancement d'une requête permettant de récupérer tout le code source de l'url à scraper

In [14]:
url = "https://professeur-layton.fandom.com/fr/wiki/La_travers%C3%A9e_(1)"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

###### Afficher le code source de la page

In [15]:
#print(soup.prettify())

###### Récupération des éléments Web à partir du soup.prettify

In [16]:
title = soup.find('meta', attrs={'property': "og:title"})
title = title.get("content")
print(title)
print("\n")

url_enigme = soup.find('meta', attrs={'property': "og:url"})
print(url_enigme.get("content"))

# Trouver la balise <tr> contenant "Professeur Layton et l'Étrange Village"
layton_row = soup.find('a', title="Professeur Layton et l'Étrange Village").find_parent('tr')
# Trouver la balise <tr> suivante
numero_row = layton_row.find_next_sibling('tr')
# Extraire le contenu de la balise <td> contenant le numéro
numero = numero_row.find('td').text.strip()
print(numero)

print("\n")
start_element = soup.find('span', id='Énoncé').find_parent('h2')

# Initialize an empty list to collect the text
text_list = []

# Iterate over all next siblings of the start element
for sibling in start_element.find_next_siblings():
    if sibling.name == 'h2':  # Stop if we reach another h2 element
        break
    if sibling.name in ['p', 'ul']:  # Collect text from p and ul elements
        text_list.append(sibling.get_text())

# Join the collected text into a single string
enigme_enonce = "\n".join(text_list)

# Print the resulting string
print(enigme_enonce)

##########

start_element = soup.find('span', id='Résolution').find_parent('h3')

# Initialize an empty list to collect the text
text_list = []

# Iterate over all next siblings of the start element
for sibling in start_element.find_next_siblings():
    if sibling.name == 'h3':  # Stop if we reach another h2 element
        break
    if sibling.name in ['p', 'ul']:  # Collect text from p and ul elements
        text_list.append(sibling.get_text())

# Join the collected text into a single string
reponse = " ".join(text_list)
print(reponse)

La traversée (1)


https://professeur-layton.fandom.com/fr/wiki/La_travers%C3%A9e_(1)
007


Vous devez amener les trois loups et les trois poussins sur l'autre rive. Attention à bien respecter les règles suivantes :

Le radeau ne peut accueillir plus de deux animaux.
Vous ne pouvez pas déplacer le radeau s'il est vide.
S'il y a plus de loups que de poussins sur une des rives, les loups mangeront les pauvres volatiles sans défense et il vous faudra recommencer depuis le début.
Vous n'avez pas de limite de déplacement, mais il est possible de résoudre cette énigme en 11 déplacements.

Beau travail !
 Cette énigme peut être résolue en 11 déplacements. Combien en avez-vous fait ?
 Il existe de nombreuses variations de ce problème. On a retrouvé de telles énigmes dans des écrits vieux de plus de mille ans.



In [18]:
# Initialiser le dictionnaire de listes
data = {
    'title': title,
    'number': numero,
    'description': enigme_enonce,
    'solution': reponse
}
# Afficher le dictionnaire
print(data)

{'title': 'La traversée (1)', 'number': '007', 'description': "Vous devez amener les trois loups et les trois poussins sur l'autre rive. Attention à bien respecter les règles suivantes\xa0:\n\nLe radeau ne peut accueillir plus de deux animaux.\nVous ne pouvez pas déplacer le radeau s'il est vide.\nS'il y a plus de loups que de poussins sur une des rives, les loups mangeront les pauvres volatiles sans défense et il vous faudra recommencer depuis le début.\nVous n'avez pas de limite de déplacement, mais il est possible de résoudre cette énigme en 11 déplacements.\n", 'solution': 'Beau travail\xa0!\n Cette énigme peut être résolue en 11 déplacements. Combien en avez-vous fait\xa0?\n Il existe de nombreuses variations de ce problème. On a retrouvé de telles énigmes dans des écrits vieux de plus de mille ans.\n'}


___

## Activité 2 : Généralisation à plusieurs énigmes

###### Lancement d'une requête permettant de récupérer tout le code source de l'url à scraper

In [58]:
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

###### Afficher le code source de la page

In [59]:
#print(soup.prettify())

###### Récuperer tous les liens des énigmes et n'en prendre que 5 aléatoirement

In [67]:
# Trouver tous les éléments <a> avec la classe "category-page__member-link"
links = soup.find_all('a', class_='category-page__member-link')

# Extraire les attributs href
hrefs = [link.get('href') for link in links]

# Afficher les hrefs
#print(hrefs)
#print(len(hrefs))

# Sélectionner aléatoirement 5 hrefs
hrefs = random.sample(hrefs, 5)

# Afficher les hrefs sélectionnés aléatoirement
#print(random_hrefs)
#print(len(random_hrefs))

###### Créons une fonction permettant de répéter les étapes d'extraction de données

In [62]:
def collecte_enigme(racine, href):
    url = racine+href
    data  = requests.get(url).text
    soup = BeautifulSoup(data,"html.parser")
  
    # Récupération du titre
    title = soup.find('meta', attrs={'property': "og:title"})
    title = title.get("content")

    # Récupération url
    url_enigme = soup.find('meta', attrs={'property': "og:url"})
    url_enigme = url_enigme.get("content")

    # Récupération de l'image
    src_tags = soup.find_all(src=True)
    # Extraire les valeurs de src
    src_urls = [tag['src'] for tag in src_tags]
    longest_src = max(src_urls, key=len) if src_urls else None
    image = longest_src

    # Récupération de l'énigme
    try:
        enigme_title = soup.find('span', {'class': 'mw-headline', 'id': 'Énoncé'})
        # Trouver le paragraphe suivant le titre de la section "Énoncé"
        enigme_paragraph = enigme_title.find_next('p')
        # Extraire le texte du paragraphe
        enigme = enigme_paragraph.get_text(strip=True)
        
    except :
        enigme = image
        
    # Récupération de la réponse
    try :
        
        reponse_title = soup.find('span', {'class': 'mw-headline', 'id': 'Solution'})
        reponse_paragraph = reponse_title.find_next('p')
        reponse = reponse_paragraph.get_text(strip=True)
    except :
        a_tag = soup.find('a', class_='image')
        if a_tag:
            reponse = a_tag.get('href')
        else:
            reponse ="La réponse n'a pas été loadé correctement"

    # Récupération des indices
    tabs = soup.select('ul.wds-tabs li.wds-tabs__tab a')
    contents = soup.select('div.wds-tab__content')
    # Extract each index content into a list
    indices = []
    for i, tab in enumerate(tabs):
        if i < len(contents):
            content_divs = contents[i].select('div[style*="overflow-y:auto"]')
            if content_divs:
                content = content_divs[0].get_text(strip=True)
                indices.append(content)
        
    # Append the collected data as a dictionary
    data = []
    data.append({'title': title, 'url': url_enigme, 'image': image, 'enigme': enigme, 'indices': indices,'solution': reponse})

    
    df = pd.DataFrame(data)
    return df

###### Utilisation de la fonction et stockage dans un DataFrame

In [68]:
racine = "https://professeur-layton.fandom.com"
df_final = pd.DataFrame()
for href in hrefs:
    #print(href)
    df_final = pd.concat([df_final, collecte_enigme(racine,href)], ignore_index=True)

###### Visualisation

In [69]:
pd.set_option('display.max_row', 7)
pd.set_option('display.max_column', 6)
df_final

,Title,url,Image,Enigme,Indices,Solution
0,Photo choc,https://professeur-layton.fandom.com/fr/wiki/P...,https://static.wikia.nocookie.net/layton/image...,Quelques amis ont eu l'idée folle d'aller fair...,[Il y a en tout six karts blancs.Les amis de l...,Il suffit d'entourer le kart blanc du milieu.
1,Histoire de gélules,https://professeur-layton.fandom.com/fr/wiki/H...,https://static.wikia.nocookie.net/layton/image...,Cet homme s'est fait prescrire dix gélules. À ...,[S'il veut indiquer l'ordre dans lequel il doi...,La réponse est 8.
2,Les trois parapluies,https://professeur-layton.fandom.com/fr/wiki/L...,https://static.wikia.nocookie.net/layton/image...,Ces trois jeunes filles ont déposé leur parapl...,[Ne perdez pas de vue ce que l'on vous demande...,La réponse est 0.
3,Chat balance...,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,Des chats en peluche de différentes couleurs s...,[Utilisez les exemples 1 et 2 pour simplifier ...,La balance penche vers A.
4,Trop d'anneaux,https://professeur-layton.fandom.com/fr/wiki/T...,https://static.wikia.nocookie.net/layton/image...,Ci-dessous se trouvent six anneaux.,[Essayez de visualiser à quoi ressemble une ch...,La réponse est le maillon D.


# Conclusion

Nous avons finalement réussi à se constituer un DataFrame à partir d'un site web ! 
Ce qu'il faut retenir, c'est que BeautifulSoup est très utile pour les sites dit "ouverts" et pour de la répetition massive de collecte d'infos.